In [1]:
#https://digital.nhs.uk/data-and-information/publications/statistical/health-survey-for-england/2021/health-survey-for-england-2021-data-tables

from dataclasses import dataclass

In [2]:
import pandas as pd
import altair as alt
import numpy as np
from tqdm import tqdm

import re
pd.options.display.max_columns = 10
pd.options.display.max_rows = 50
pd.set_option('display.max_colwidth', None)
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

Load the data into dataframes

In [3]:
          
tables = [{"filename": "obes-phys-acti-diet-eng-2014-tab.xls", "sheet_name":"7.6",  "rows_top": 3, "rows_bottom": 185, "columns_left": 2, "columns_right" :10},
          {"filename": "obes-phys-acti-diet-eng-2015-tab.xlsx", "sheet_name":"7.7", "rows_top": None, "rows_bottom": None, "columns_left": 3, "columns_right" :11},
          {"filename": "obes-phys-acti-diet-eng-2016-tab.xlsx", "sheet_name":"Table 7", "rows_top": None, "rows_bottom": None, "columns_left": 3, "columns_right" :11},
           {"filename": "obes-phys-acti-diet-eng-2017-tab.xlsx", "sheet_name":"Table 7", "rows_top": None, "rows_bottom": None, "columns_left": 3, "columns_right" :11},
           {"filename": "obes-phys-acti-diet-eng-2018-tab.xlsx", "sheet_name":"Table 7", "rows_top": None, "rows_bottom": None, "columns_left": 2, "columns_right" :11},
           {"filename": "Obes-phys-acti-diet-eng-tab-2019.xlsx", "sheet_name":"Table 2.4", "rows_top": None, "rows_bottom": None, "columns_left": 2, "columns_right" :11},
           ]
df_columns = ["region", "admissions_total",
                          "admissions_count_men", "admissions_count_women",
                          "adm_per_100_000_all", "adm_per_male", "adm_per_female", "year"]
#"South Central"
region_list = ["East Midlands", "East of England", "West Midlands",
               "North East", "North West", "North Yorkshire","London", "South East", "South West", "West Sussex", "Yorkshire and the Humber"]
print(len(region_list))

11


In [4]:
@dataclass
class Obesity:
    def collect_data(self):
        df_list = []
        for t in tables:
            filename = t['filename']
            year = re.search(r'\d{4}', filename).group()
            df = pd.read_excel(filename, sheet_name=t['sheet_name']).dropna(axis=0, how="all")
            df = df.dropna(axis=1, how="all")
            df['year'] = year
            
            df = df.iloc[t['rows_top']: t['rows_bottom'], t['columns_left']: t['columns_right']]
            df = df.dropna(axis=0)
            df.columns = df_columns
            df['region'] = df['region'].apply(lambda x: x.replace(" PCT", ""))            
            df['region'] = df['region'].apply(lambda x: x.title())
            df['region'] = df['region'].str.replace(r'\d+', '', regex=True)
            df['region'] = df['region'].apply(lambda x: x.replace(" Borough Council (Unitary)", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace(" Metropolitan Borough Council", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace("  County Council", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace("  Council (Unitary)", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace("  City Council", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace("  Borough Council", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace(" Council", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace(" (Unitary)", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace(" Borough", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace(" Care Trust", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace(" Plus", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace(" County", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace(" City", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace(" Metropolitan District", ""))
            df['region'] = df['region'].apply(lambda x: x.replace(", Of", ""))
            df['region'] = df['region'].apply(lambda x: x.replace(" Teaching", "")) 
            df['region'] = df['region'].apply(lambda x: x.replace("Kensington & Chelsea", "Kensington And Chelsea"))
            df['region'] = df['region'].apply(lambda x: x.replace("Leicestershire & Rutland", "Leicestershire"))
            df['region'] = df['region'].apply(lambda x: x.replace("Newcastle upon Tyne", "Newcastle-upon-Tyne"))
            df['region'] = df['region'].apply(lambda x: x.replace("Stoke on Trent", "Stoke-on-Trent"))    
            df['region'] = df['region'].apply(lambda x: x.replace("Redcar &Cleveland", "Redcar and Cleveland"))
            df['region'] = df['region'].apply(lambda x: x.replace("Richmond & Twickenham", "Richmond Upon Thames"))
            df['region'] = df['region'].apply(lambda x: x.replace("Tameside & Glossop", "Tameside"))
            df['region'] = df['region'].apply(lambda x: x.replace("Telford & Wrekin", "Telford And Wrekin"))
            df['region'] = df['region'].apply(lambda x: x.replace("Sutton & Merton", "Sutton")) 
            df['region'] = df['region'].apply(lambda x: x.replace("Barking & Dagenham", "Barking And Dagenham"))     
            df['region'] = df['region'].apply(lambda x: x.replace("Bath & North East Somerset", "Bath And North East Somerset"))   
            df['region'] = df['region'].apply(lambda x: x.replace("Brighton & Hove", "Brighton And Hove"))     
            df['region'] = df['region'].apply(lambda x: x.replace("London Of ", ""))
            df['region'] = df['region'].apply(lambda x: x.replace("Hammersmith & Fulham", "Hammersmith And Fulham")) 
            df['region'] = df['region'].apply(lambda x: x.replace("Hammersmith & Fulham", "Hammersmith And Fulham")) 
            df['region'] = df['region'].apply(lambda x: x.replace("Redcar & Cleveland", "Redcar And Cleveland")) 
            df['region'] = df['region'].apply(lambda x: x.replace("Newcastle Upon Tyne", "Newcastle-Upon-Tyne")) 
            df['region'] = df['region'].apply(lambda x: x.replace("Stoke On Trent", "Stoke-On-Trent")) 
            df['region'] = df['region'].apply(lambda  x: x.replace("The Of The Isles of Scilly", "Isles Of Scilly"))  
            df['region'] = df['region'].apply(lambda x: x.strip())
            
            df_list.append(df)
        return df_list
 
    
    # Example usage:
instance = Obesity()
result = instance.collect_data()

result[0]

,region,admissions_total,admissions_count_men,admissions_count_women,adm_per_100_000_all,adm_per_male,adm_per_female,year
5,England,292404,99579,192795,550.592332,381.04459,714.743642,2014
7,North East,17409,6294,11115,670.494727,495.747473,837.702727,2014
8,North East,17409,6294,11115,670.494727,495.747473,837.702727,2014
9,County Durham,2240,850,1390,436.652281,338.418907,530.886925,2014
10,Darlington,283,99,184,268.033035,192.382433,339.960092,2014
...,...,...,...,...,...,...,...,...
181,Somerset,3565,905,2660,670.640975,348.965443,977.068281,2014
182,South Gloucestershire,1396,530,866,529.958203,405.511901,652.51134,2014
183,Swindon,834,333,501,388.082065,309.58601,466.741196,2014
184,Torbay,1243,478,765,947.459087,756.257317,1125.215115,2014


In [5]:
result[0].to_csv("obesity.csv")


#for r in result:
#    print(r.shape)

    
    


    



In [6]:
result[0]

,region,admissions_total,admissions_count_men,admissions_count_women,adm_per_100_000_all,adm_per_male,adm_per_female,year
5,England,292404,99579,192795,550.592332,381.04459,714.743642,2014
7,North East,17409,6294,11115,670.494727,495.747473,837.702727,2014
8,North East,17409,6294,11115,670.494727,495.747473,837.702727,2014
9,County Durham,2240,850,1390,436.652281,338.418907,530.886925,2014
10,Darlington,283,99,184,268.033035,192.382433,339.960092,2014
...,...,...,...,...,...,...,...,...
181,Somerset,3565,905,2660,670.640975,348.965443,977.068281,2014
182,South Gloucestershire,1396,530,866,529.958203,405.511901,652.51134,2014
183,Swindon,834,333,501,388.082065,309.58601,466.741196,2014
184,Torbay,1243,478,765,947.459087,756.257317,1125.215115,2014


In [7]:
df = pd.concat([result[0], result[1], result[2],
                result[5],result[4], result[5]])


df['admissions_total'] = df['admissions_total'].astype(int)
df['admissions_count_men'] = pd.to_numeric(df['admissions_count_men'], errors=
'coerce')
df['admissions_count_women'] = pd.to_numeric(df['admissions_count_women'], errors=
'coerce')
df['adm_per_100_000_all'] = pd.to_numeric(df['adm_per_100_000_all'], errors=
'coerce')
df['adm_per_female'] = pd.to_numeric(df['adm_per_female'], errors=
'coerce')
df['adm_per_male'] = pd.to_numeric(df['adm_per_male'], errors=
'coerce')
df['year'] = pd.to_numeric(df['year'], errors=
'coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 983 entries, 5 to 181
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   region                  983 non-null    object 
 1   admissions_total        983 non-null    int64  
 2   admissions_count_men    982 non-null    float64
 3   admissions_count_women  982 non-null    float64
 4   adm_per_100_000_all     981 non-null    float64
 5   adm_per_male            980 non-null    float64
 6   adm_per_female          980 non-null    float64
 7   year                    983 non-null    int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 69.1+ KB


In [8]:
def test_top(s):
    """"Create the top tiers column which suggests if df is either major region or England. This will be used to create the subset of local authorities"""
    
    if s in region_list or s == "England":
        return True
    else:
        return np.NaN
df['top_tier'] = df['region'].apply(test_top)    


In [9]:
df[df['region']=="England"]['admissions_count_women'].sum()

2068246.0

In [10]:

df_england = df.loc[df['region'] == "England"]
df_england_plot = alt.Chart(df_england).mark_bar().encode(
    alt.X("year:O", title="Year"),
    alt.Y("admissions_total:Q", title="Admissions"),
    tooltip = alt.Tooltip("admissions_total", format=",")

).properties(title="Hospital admissions due to obesity per year in the UK", width=100)
df_england_plot

alt.Chart(...)

In [11]:


"""This line creates a boolean Series named cond where each value is True if the corresponding value in the top_tier column of the DataFrame df is equal to True, and False otherwise. In other words, it identifies rows where the top_tier column is True."""

cond = df['top_tier'].eq(True)

"""This line does several things:

df['region'].where(cond): This returns a Series where values from the region column are retained only for rows where cond is True. For rows where cond is False, the values will be replaced with NaN.

.ffill(): This method is called on the result of the previous operation. It stands for "forward fill" and it fills NaN values with the previous non-NaN value in the Series. This means that if a row has a NaN value because its top_tier value was False, it will now get the value from the last row where top_tier was True.

.where(~cond): This is applied to the result of the ffill() operation. It retains values only for rows where cond is False (because of the ~, which is a bitwise negation operator). For rows where cond is True, the values will be replaced with NaN.

The final result of this operation is a Series where:

Rows with top_tier set to True will have NaN values.
Rows with top_tier set to False will have values from the region column, but these values will be the same as the last row where top_tier was True.
Assigning the Result:
The resulting Series is assigned to both a new column in the DataFrame df named result and a separate variable named df_regional_level."""
df_regional_level = df['result'] = df['region'].where(cond).ffill().where(~cond)
#df_regional_level

In [12]:
df_major = df.loc[df['region'].isin(region_list)]
df_major[['admissions_total', 
          "admissions_count_men",
          "admissions_count_women"]].sum()

admissions_total          2772182.0
admissions_count_men       944883.0
admissions_count_women    1827127.0
dtype: float64

In [13]:
df_major.columns

Index(['region', 'admissions_total', 'admissions_count_men',
       'admissions_count_women', 'adm_per_100_000_all', 'adm_per_male',
       'adm_per_female', 'year', 'top_tier', 'result'],
      dtype='object')

In [14]:
df_major.loc[df_major['region'] == "North West"]['admissions_total'].sum()


523967

In [15]:
regional = alt.Chart(df_major).mark_line(size=5, point=True).encode(
    alt.X("year:O", title="Year"),
    alt.Y("admissions_total:Q", title="Total admissions"),
    color=alt.Color("region:N", title="Region"), 

    tooltip=[
        alt.Tooltip("region:N", title="Region"),
        alt.Tooltip("admissions_total:Q", title="Admissions", format=",")
    ]).properties(width=100)
#.configure_legend(orient='left').properties(title="Obesety-related hospital admissions by UK regions", width=200)


regional

alt.Chart(...)

In [16]:
(df_england_plot | regional)

alt.HConcatChart(...)

Sept 1, 2023: So far the numbers and the charts seem reasonable. 

In [17]:
#Droppin the  boolean column as it has served its purpose it also remooves the result dropna()
local_results = df.drop("top_tier", axis=1).dropna()

In [18]:
local_results.head()

,region,admissions_total,admissions_count_men,admissions_count_women,adm_per_100_000_all,adm_per_male,adm_per_female,year,result
9,County Durham,2240,850.0,1390.0,436.652281,338.418907,530.886925,2014,North East
10,Darlington,283,99.0,184.0,268.033035,192.382433,339.960092,2014,North East
11,Gateshead,1229,396.0,833.0,613.429565,403.953851,814.128501,2014,North East
12,Hartlepool,484,163.0,321.0,525.584224,364.164433,678.245436,2014,North East
13,Middlesbrough,422,170.0,252.0,304.983811,251.141215,356.551636,2014,North East


In [19]:
#Rename the remaining columns

In [20]:
local_results.columns = ['Region', 'Total admissions',
                         'Admissions count: men',
                         'Admissions count: women', 
                         'Admissions per 100,000 population', 
                         'Admissions per 100,000 population: men',
                         'Admissions per 100,000 population: women', 
                         'Year', 'Major region']


melt the wide data into long form for Altair use

In [21]:

melted_df = pd.melt(local_results, id_vars=['Region', "Major region", "Year"])
melted_df.columns


melted_df.head()

,Region,Major region,Year,variable,value
0,County Durham,North East,2014,Total admissions,2240.0
1,Darlington,North East,2014,Total admissions,283.0
2,Gateshead,North East,2014,Total admissions,1229.0
3,Hartlepool,North East,2014,Total admissions,484.0
4,Middlesbrough,North East,2014,Total admissions,422.0


In [22]:
#This tests whether on a par with expected


melted_df.loc[melted_df['variable'] == "Total admissions"]['value'].sum()

3791053.0

In [23]:
#This tests that England is not i the list
melted_df[melted_df['Region'] == "England"]
filtered_df = melted_df


Create a plot for gender

In [24]:
women_df = melted_df[melted_df['variable'] == 'Admissions count: women']
men_df = melted_df[melted_df['variable'] == 'Admissions count: men']

# Create separate charts for women and men
women_chart = alt.Chart(women_df).mark_bar(opacity=0.7).encode(
    x=alt.X("Year:O", title="Year"),
    y=alt.Y("value:Q", title="Admissions count", stack=None),
    color=alt.Color("variable:N", title="Gender"),
    tooltip=[
        alt.Tooltip("variable:N", title="Gender"),
        alt.Tooltip("value:Q", title="Admissions count",format=
",")
    ]
).properties(
    title="Admissions for men and women",
    width=50
)

men_chart = alt.Chart(men_df).mark_bar(opacity=1).encode(
    x=alt.X("Year:O", title="Year"),
    y=alt.Y("value:Q", title="Admissions count", stack=None),
    color=alt.Color("variable:N", title="Gender"),
    tooltip=[
        alt.Tooltip("variable:N", title="Gender"),
        alt.Tooltip("value:Q", title="Admissions count",format=
",")]
).properties(width=100)
# Overlay the men_chart on top of the women_chart
overlay_chart = women_chart + men_chart
overlay_chart


ValueError: There are inconsistent values [50, 100] for width

In [36]:
# Create dropdown for Major region
major_region_dropdown = alt.binding_select(options=melted_df['Major region'].unique().tolist(), name="Major region: ")
major_region_selection = alt.selection_single(fields=['Major region'], bind=major_region_dropdown, init={"Major region": "West Midlands"})

# Create dropdown for variable
variable_dropdown = alt.binding_select(options=melted_df['variable'].unique().tolist(), name="Category: ")
variable_selection = alt.selection_single(fields=['variable'], bind=variable_dropdown, init={"variable": "Total admissions"})

# Sort the data
sorted_data = melted_df.sort_values(by='Year')

# Main chart
main_chart = alt.Chart(sorted_data).mark_line(point=True).encode(
    x='Year:O',
    y='mean(value):Q',
    color=alt.Color('Region:N', legend=None),  # Set legend to None
    tooltip=["Major region", "Region", "mean(value):Q"]
).add_selection(
    major_region_selection
).transform_filter(
    major_region_selection
).add_selection(
    variable_selection
).transform_filter(
    variable_selection
).properties(
    width=450, 
    height=600
)

# Secondary chart for Region selection
region_chart = alt.Chart(sorted_data).mark_bar().encode(
    y='Region:N',
    color=alt.Color('Region:N', legend=None),  # Set legend to None
    tooltip=alt.Tooltip(["Major region", "Region"])  # Corrected tooltip placement
).add_selection(
    major_region_selection
).transform_filter(
    major_region_selection
).add_selection(
    variable_selection
).transform_filter(
    variable_selection
).properties(
    selection=alt.selection_multi(fields=['Region']),
    width=30,  
    height=600,
    title="Region"
    
).interactive()
((df_england_plot | overlay_chart| regional)&(main_chart | region_chart))

/Users/davidelks/opt/anaconda3/lib/python3.9/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/davidelks/opt/anaconda3/lib/python3.9/site-packages/altair/vegalite/v5/api.py:388: AltairDeprecationWarning: Use 'value' instead of 'init'.
  warnings.warn(


TypeError: altair.vegalite.v5.schema.core.SelectionParameter() got multiple values for keyword argument 'value'

Now let's put in the data from the Food Standards Agency


In [37]:
from bs4 import BeautifulSoup
import requests
base_url = "https://ratings.food.gov.uk/open-data"
data = requests.get(base_url)
soup = BeautifulSoup(data.text, 'html.parser')


In [38]:
# Find all links in the HTML
links = soup.find_all('a')

# Initialize an empty list to store the XML URLs
xml_urls = []

# Loop through all links and filter those ending with .xml
for link in tqdm(links):
    url = link.get('href')
    if url and "FHRS" in url and url.endswith('.xml'):
        xml_urls.append(url)

# Print the list of XML URLs
print(xml_urls)


100%|██████████| 516/516 [00:00<00:00, 249972.38it/s]

['/api/open-data-files/FHRS297en-GB.xml', '/api/open-data-files/FHRS109en-GB.xml', '/api/open-data-files/FHRS701en-GB.xml', '/api/open-data-files/FHRS110en-GB.xml', '/api/open-data-files/FHRS227en-GB.xml', '/api/open-data-files/FHRS111en-GB.xml', '/api/open-data-files/FHRS228en-GB.xml', '/api/open-data-files/FHRS155en-GB.xml', '/api/open-data-files/FHRS027en-GB.xml', '/api/open-data-files/FHRS112en-GB.xml', '/api/open-data-files/FHRS702en-GB.xml', '/api/open-data-files/FHRS113en-GB.xml', '/api/open-data-files/FHRS114en-GB.xml', '/api/open-data-files/FHRS156en-GB.xml', '/api/open-data-files/FHRS028en-GB.xml', '/api/open-data-files/FHRS157en-GB.xml', '/api/open-data-files/FHRS302en-GB.xml', '/api/open-data-files/FHRS115en-GB.xml', '/api/open-data-files/FHRS029en-GB.xml', '/api/open-data-files/FHRS229en-GB.xml', '/api/open-data-files/FHRS116en-GB.xml', '/api/open-data-files/FHRS158en-GB.xml', '/api/open-data-files/FHRS030en-GB.xml', '/api/open-data-files/FHRS299en-GB.xml', '/api/open-data

In [39]:
json_urls = ["https://ratings.food.gov.uk"+xml.replace("xml", "json")for xml in xml_urls]

In [40]:
js_len = len(json_urls)
print(js_len)

def fetch_data(url, js_len):
    json = pd.read_json(url, orient="records")  # Removed quotes around url
    df = pd.DataFrame(json['FHRSEstablishment'][0])
    js_len = js_len - 1
    return df

results = [fetch_data(j, js_len) for j in tqdm(json_urls)]


385


100%|██████████| 385/385 [03:45<00:00,  1.71it/s]


I don't want to wait four minutes

In [41]:
fs =pd.concat(results)
    

In [42]:
fs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 628676 entries, 0 to 1047
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   FHRSID                      628676 non-null  int64 
 1   LocalAuthorityBusinessID    628676 non-null  object
 2   BusinessName                628676 non-null  object
 3   BusinessType                628676 non-null  object
 4   BusinessTypeID              628676 non-null  int64 
 5   AddressLine1                420371 non-null  object
 6   AddressLine2                461429 non-null  object
 7   AddressLine3                416029 non-null  object
 8   AddressLine4                272331 non-null  object
 9   PostCode                    524495 non-null  object
 10  RatingValue                 628676 non-null  object
 11  RatingKey                   628676 non-null  object
 12  RatingDate                  580819 non-null  object
 13  LocalAuthorityCode          628

In [43]:
fs.shape

(628676, 22)

In [44]:
fs.columns

Index(['FHRSID', 'LocalAuthorityBusinessID', 'BusinessName', 'BusinessType',
       'BusinessTypeID', 'AddressLine1', 'AddressLine2', 'AddressLine3',
       'AddressLine4', 'PostCode', 'RatingValue', 'RatingKey', 'RatingDate',
       'LocalAuthorityCode', 'LocalAuthorityName', 'LocalAuthorityWebSite',
       'LocalAuthorityEmailAddress', 'Scores', 'SchemeType',
       'NewRatingPending', 'Geocode', 'RightToReply'],
      dtype='object')

In [45]:
fs.BusinessType.unique()


array(['Hospitals/Childcare/Caring Premises', 'Other catering premises',
       'Restaurant/Cafe/Canteen', 'Retailers - other',
       'School/college/university',
       'Retailers - supermarkets/hypermarkets', 'Mobile caterer',
       'Pub/bar/nightclub', 'Hotel/bed & breakfast/guest house',
       'Manufacturers/packers', 'Takeaway/sandwich shop',
       'Farmers/growers', 'Distributors/Transporters',
       'Importers/Exporters'], dtype=object)

In [46]:
# Convert 'RatingDate' to datetime format
fs['RatingDate'] = pd.to_datetime(fs['RatingDate'])

# Convert 'RatingValue' to integer (assuming 'df' should be 'fs')
fs['RatingValue'] = pd.to_numeric(fs['RatingValue'], errors='coerce').fillna(0).astype(int)

# Extract the year from 'RatingDate' and fill NA values with 0
fs['Year'] = fs['RatingDate'].dt.year.fillna(0).astype(int)

# Filter the DataFrame based on the year range
ffs = fs.loc[(fs['Year'] >= 2013) & (fs['Year'] <= 2019)]

# Display information about the filtered DataFrame
ffs.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 134995 entries, 2 to 1021
Data columns (total 23 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   FHRSID                      134995 non-null  int64         
 1   LocalAuthorityBusinessID    134995 non-null  object        
 2   BusinessName                134995 non-null  object        
 3   BusinessType                134995 non-null  object        
 4   BusinessTypeID              134995 non-null  int64         
 5   AddressLine1                88891 non-null   object        
 6   AddressLine2                96178 non-null   object        
 7   AddressLine3                88050 non-null   object        
 8   AddressLine4                55716 non-null   object        
 9   PostCode                    110405 non-null  object        
 10  RatingValue                 134995 non-null  int64         
 11  RatingKey                   134995 non-nu

In [47]:
numbers_df_ct = pd.DataFrame(ffs['RatingValue'].value_counts()).reset_index()
numbers_df_per = pd.DataFrame(ffs['RatingValue'].value_counts(normalize=True)).reset_index()
numbers_df_ct.columns = numbers_df_per.columns = ["Rating", "Numbers"]
percentage_df = pd.DataFrame(ffs['RatingValue'].value_counts()).reset_index()
percentage_per = pd.DataFrame(ffs['RatingValue'].value_counts(normalize=True)).reset_index()
percentage_df.columns = percentage_per =["Rating", "Percentage"]

Index(['FHRSID', 'LocalAuthorityBusinessID', 'BusinessName', 'BusinessType',
       'BusinessTypeID', 'AddressLine1', 'AddressLine2', 'AddressLine3',
       'AddressLine4', 'PostCode', 'RatingValue', 'RatingKey', 'RatingDate',
       'LocalAuthorityCode', 'LocalAuthorityName', 'LocalAuthorityWebSite',
       'LocalAuthorityEmailAddress', 'Scores', 'SchemeType',
       'NewRatingPending', 'Geocode', 'RightToReply', 'Year'],
      dtype='object')

In [ ]:
ffs.value_counts()



TypeError: unhashable type: 'dict'